In [1]:
from PIL import Image
import os
import pathlib
import cv2
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import random
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle as sk_shuffle
from skimage.util import random_noise
import time
import os
from torch.utils import data
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

from cal_dl_oldskool import RetraceDataLoader, retrace_parser, retrace_parser_synth
# from calculus_dataloader import RetraceDataLoader, CalculusBase

from torch.utils.data.sampler import SubsetRandomSampler, SequentialSampler
from custom_unets import NestedUNet, U_Net, DeepNestedUNet
from sync_batchnorm import SynchronizedBatchNorm2d, DataParallelWithCallback, convert_model
# from kornia.losses import FocalLoss
from pywick.losses import BCEDiceFocalLoss, BinaryFocalLoss

import glob2
import pdb
import ipdb

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
import torchvision.models as models
import torch.nn as nn

rohan_unet = DeepNestedUNet(1,1)
if torch.cuda.device_count() > 0:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    rohan_unet = nn.DataParallel(rohan_unet)
rohan_unet = rohan_unet.to(device)
rohan_unet = convert_model(rohan_unet)
rohan_unet = rohan_unet.to(device)
# rohan_unet.load_state_dict(torch.load('/home/rohan/prior_seg/models/calculus_full1/calculus_256_model_epoch_16.0_f1_0.6215.pth'))

Let's use 2 GPUs!


In [4]:
pretrained_dict = torch.load('/home/rohan/prior_seg/models/prior_256/256_model_epoch_65.0_f1_0.8814.pth')
print('Weight dict before = {} weights'.format(len(pretrained_dict)))
model_dict = rohan_unet.state_dict()

# filter unnecessary keys
pretrained_dict = {k: v for k, v in pretrained_dict.items() if
                       (k in model_dict) and (model_dict[k].shape == pretrained_dict[k].shape)}
print('Weight dict after = {} weights'.format(len(pretrained_dict)))
# 2. overwrite entries in the existing state dict
model_dict.update(pretrained_dict) 
# 3. load the new state dict
rohan_unet.load_state_dict(model_dict)

Weight dict before = 296 weights
Weight dict after = 294 weights


<All keys matched successfully>

In [5]:
# for module in list(rohan_unet.children()):
#     for layer in list(module.children())[:-1]:
#         for param in layer.parameters():
#             param.requires_grad = False

# for module in list(rohan_unet.children()):
#     for layer in list(module.children()):
#         for param in layer.parameters():
#             print(layer, param.requires_grad)

In [6]:
from torchsummary import summary
# summary(rohan_unet, input_size=(1,256,256))

In [7]:
# random_seed= 42
# mark4 = time.time()
# root_dir='/home/rohan/Datasets/prior_clean/train'
# calculus_clean_train = CalculusBase(root_dir, oversample=2, issynthetic=False, test_size=0.1,im_range='all', isTrain=True)
# calculus_clean_val = CalculusBase(root_dir, oversample=2, issynthetic=False, test_size=0.1,im_range='all', isTrain=False)
# print('Real dataload time {:.6f}'.format(time.time() - mark4))

# mark3 = time.time()
# synth_root_dir='/home/rohan/Datasets/synthetic_prior_clean/train'
# calculus_synthetic_train = CalculusBase(synth_root_dir, oversample=1, issynthetic=True, test_size=0.05,im_range=[0,88000], isTrain=True)
# calculus_synthetic_val = CalculusBase(synth_root_dir, oversample=1, issynthetic=True, test_size=0.05,im_range=[0,88000], isTrain=False)
# print('Synthetic dataload time {:.6f}'.format(time.time() - mark3))

root_dir = '/home/rohan/Datasets/prior_clean/train/'
syn_root_dir = '/home/rohan/Datasets/synthetic_prior_clean/train/'

# prior_data = RetraceDataLoader(root_dir, syn_root_dir, length = 100)
teeth_dataset = RetraceDataLoader(root_dir=root_dir,
                                  root_dir_synth=syn_root_dir,
                                  image_size=(256,256),
                                  length = 'all',# pass 'all' for all
                                  transform=None)


Dataset length:  52131
Loaded dataset length: 52131
Dataset length synthetic:  261030


In [8]:
# train_dataset = RetraceDataLoader(calculus_clean_train,
#                                            calculus_synthetic_train,
#                                   image_size=(384,384),
#                                   length = 'all',# pass 'all' for all
#                                   crop = True,
#                                   transform=None,
#                                  mode = 'train')


# val_dataset = RetraceDataLoader(calculus_clean_val,
#                                            None,
#                                   image_size=(384,384),
#                                   length = 'all',# pass 'all' for all
#                                   crop = True,
#                                   transform=None,
#                                mode = 'val')

In [9]:
from torch.utils.data.sampler import SubsetRandomSampler

validation_split = .1
shuffle_dataset = True
random_seed= 42
batch_size = 48

# Creating data indices for training and validation splits:
dataset_size = len(teeth_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

def worker_init_fn(worker_id):                                                          
    np.random.seed(np.random.get_state()[1][0] + worker_id)

trainloader = torch.utils.data.DataLoader(
    teeth_dataset,
    batch_size=batch_size,
    num_workers=4,
    shuffle=False,
    sampler=train_sampler,
    worker_init_fn=worker_init_fn,
    pin_memory = True,
    drop_last =True
)
valloader = torch.utils.data.DataLoader(
    teeth_dataset,
    batch_size=batch_size,
    num_workers=4,
    shuffle=False,
    sampler=valid_sampler,
    worker_init_fn=worker_init_fn,
    pin_memory = True,
    drop_last =True
)
print ('Train size: ', len(trainloader))
print ('Validation size: ', len(valloader))

Train size:  977
Validation size:  108


In [10]:
import time
import copy
import pdb
import pandas as pd

dataloaders = {'train': trainloader,'val':valloader}
dataset_sizes = {'train':len(trainloader), 'val':len(valloader)}


SMOOTH = 1e-6


def dice_loss(input, target):
    smooth = SMOOTH
    
    iflat = input.view(-1)
    tflat = target.view(-1)
    intersection = (iflat * tflat).sum()
    
    return 1 - ((2. * intersection + smooth) /
              (iflat.sum() + tflat.sum() + smooth))

def dice_score(input, target):
    smooth = SMOOTH
#     print(input.shape)
#     ipdb.set_trace()
    iflat = input.view(-1)
    tflat = target.view(-1)
    intersection = (iflat * tflat).sum()
    
    return ((2. * intersection + smooth) /
              (iflat.sum() + tflat.sum() + smooth))

def dice_per_channel(inputs, target):
    
    dice_ch = 0.0
    for i in range(0, inputs.shape[1]):
        inp = inputs[:,i,:,:]
        inp = inp.contiguous()
        targs = target[:,i,:,:]
        targs = targs.contiguous()
        dice_chl = dice_score(inp,targs)
        dice_ch +=dice_chl
    
    return dice_ch / (inputs.shape[1]-1)

def dice_per_image(inputs, target):
    
    dice_img = 0.0
    for i in range(0, inputs.shape[0]):
        inp = inputs[i,:,:,:]
        inp = inp.contiguous()
        targs = target[i,:,:,:]
        targs = targs.contiguous()
        dice_im = dice_score(inp,targs)
        dice_img +=dice_im
    
    return dice_img / (inputs.shape[0]-1)


def train_model(model, criterion, optimizer, scheduler, writer, num_epochs=15):
    start = time.time()
    save_dict={}
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 10.0
    best_iou = 0.0
    best_f1 = 0.0
    best_f1_ch = 0.0
    best_f1_img = 0.0

    for epoch in range(num_epochs):
        ep_start = time.time()
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        lrate = scheduler.get_lr()[0]
        writer.add_scalar('Learning Rate', lrate, epoch)
        print('LR {:.5f}'.format(lrate))
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
            running_loss = 0.0
            running_ious = 0.0
            running_f1 = 0.0
            running_f1_ch = 0.0
            running_f1_img = 0.0

            # Iterate over data.
            for data in dataloaders[phase]:
#                 ipdb.set_trace()
                inputs = data['image'][:,:,:,:]
                labels = data['masks'][:,:,:,:]
#               labels = labels.unsqueeze(0)
#                 labels = labels.float()
                
                inputs = inputs.to(device)
                labels = labels.to(device)
                labels = labels.type(torch.cuda.FloatTensor)
                 # zero the parameter gradients
                optimizer.zero_grad()
#               torch.autograd.set_detect_anomaly(True)
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    
                    outputs = model(inputs)
                    loss = criterion(outputs,labels)
                    preds = torch.sigmoid(outputs)
                    
#                     bce = criterion(preds, labels)
#                     diceloss = dice_loss(preds,labels)
#                     loss = fl * 0.8 + diceloss * (1 - 0.8)
                    
                    bin_preds = preds.clone().detach()
                    bin_preds[bin_preds<=0.5]= 0.0
                    bin_preds[bin_preds>0.5]= 1.0
                    
#                     ipdb.set_trace()
                    f1 = dice_score(bin_preds, labels)
                    f1_img = dice_per_image(bin_preds,labels)
                    
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    
                    # statistics
                    running_loss += loss.data.cpu().numpy() # * inputs.size(0)
                    running_f1 += f1
                    running_f1_img += f1_img
                    
            torch.cuda.empty_cache()
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_f1 = running_f1 / dataset_sizes[phase]
            epoch_f1_img = running_f1_img / dataset_sizes[phase]

            if phase == 'train':
                writer.add_scalar('Loss/train', epoch_loss, epoch)
                writer.add_scalar('Hard_Dice/train', epoch_f1, epoch)
                writer.add_scalar('Hard_Dice_per_image/train', epoch_f1_img, epoch)
            else:
                writer.add_scalar('Loss/val', epoch_loss, epoch)
                writer.add_scalar('Hard_Dice/val', epoch_f1, epoch)
                writer.add_scalar('Hard_Dice_per_image/val', epoch_f1_img, epoch)

            print('{} Loss: {:.5f} F1: {:.4f} F1/img: {:.4f}'.format(phase, epoch_loss, epoch_f1, epoch_f1_img))#F1/ch: {:.4f} epoch_f1_ch,

            # deep copy the model
            if phase == 'val' and epoch_f1 > best_f1:
                best_loss = epoch_loss
                best_f1 = epoch_f1
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(best_model_wts, '/home/rohan/prior_seg/models/calculus_onlyfocal/calculus_256_model_epoch_{:.1f}_f1_{:.4f}.pth'.format(epoch, best_f1))
            writer.add_scalar('Hard_Dice/best_val', best_f1, epoch)
            

        print('Epoch completed in {:.4f} seconds'.format(time.time()-ep_start))
        torch.cuda.empty_cache()
        

    time_elapsed = time.time() - start
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best Val F1: {:4f}'.format(best_f1))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
from torch.autograd import Function
from torch.autograd import Variable


criterion = BinaryFocalLoss(gamma=2.0, alpha=0.25) #torch.nn.BCELoss()
print("Focal Loss alpha = {:.2f} gamma = {:.1f}".format(criterion.alpha, criterion.gamma))
optimizer = optim.Adam(rohan_unet.parameters(), lr=0.00035)
writer = SummaryWriter(log_dir='/home/rohan/prior_seg/logs/calculus_onlyfocal', filename_suffix = '_calculus_focal')
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.95)
# rohan_unet.load_state_dict(torch.load('/home/rohan/caries_seg/models/test_model/128_nestedunet_model_epoch_0.0000_f1_0.1858.pth'))
model_trained = train_model(rohan_unet, criterion, optimizer, exp_lr_scheduler, writer = writer, num_epochs=50)

Focal Loss alpha = 0.25 gamma = 2.0
Epoch 0/49
----------
LR 0.00035
train Loss: 0.00044 F1: 0.1056 F1/img: 0.7526
val Loss: 0.00019 F1: 0.1081 F1/img: 0.7685
Epoch completed in 501.1740 seconds
Epoch 1/49
----------
LR 0.00035
train Loss: 0.00017 F1: 0.3462 F1/img: 0.7708
val Loss: 0.00015 F1: 0.3272 F1/img: 0.7900
Epoch completed in 497.6765 seconds
Epoch 2/49
----------
LR 0.00035
train Loss: 0.00015 F1: 0.4148 F1/img: 0.7859
val Loss: 0.00013 F1: 0.4574 F1/img: 0.7857
Epoch completed in 497.4518 seconds
Epoch 3/49
----------
LR 0.00032
train Loss: 0.00013 F1: 0.4590 F1/img: 0.7975
val Loss: 0.00011 F1: 0.4900 F1/img: 0.7972
Epoch completed in 498.3475 seconds
Epoch 4/49
----------
LR 0.00033
